In [34]:
import sys
sys.path.append("..")

In [35]:
import os
from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [36]:
graph.query('MATCH (n) DETACH DELETE n;')
# graph.query('drop index chunk_index;')

[]

In [37]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_experimental.llms.ollama_functions import OllamaFunctions
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)

In [38]:
# from langchain_experimental.llms.ollama_functions import OllamaFunctions
# model_name = 'qwen2:72b-instruct-q8_0'
# llm = OllamaFunctions(model=model_name, temperature=0)
# llm

In [39]:

# from langchain_community.document_loaders import PyPDFLoader
from tools.PDFTablePyPlumberLoader import PDFTablePyPlumberLoader

# from pdfminer.high_level import extract_text

# from langchain_community.document_loaders import PyPDFLoader

pages = []
dir = '../data'
for filename in os.listdir(dir):
    if filename.endswith(".pdf"):
        # if '202311RZ1A22021A11Z10000015-A' not in filename:
        #     continue
        # text = PyPDFLoader(os.path.join(dir, filename)).load()
        # print(f'text: {text}')
        loader = PDFTablePyPlumberLoader(os.path.join(dir, filename), llm)
        
        pages += [loader.load()]
    # break # too much time, only load one file
pages
# import json
# with open('data_read_from_pdf.json', 'w') as file:
#     # page_json = 
#     json.dump(pages, file, indent=4, ensure_ascii=False)

Processing documents: 100%|██████████| 5/5 [00:13<00:00,  2.79s/it]


[[Document(metadata={'source': '台灣人壽龍實在住院醫療健康保險附約.pdf', 'page_number': 1}, page_content='台灣人壽保險股份有限公司\n（以下簡稱「本公司」）\n台灣人壽龍實在住院醫療健康保險附約\n主要給付項目： 中華民國113年7月1日\n1.住院日額保險金 台壽字第1132320033號函備查\n2.住院醫療費用保險金(實支實付) 中華民國113年10月1日\n3.門診手術費用保險金(實支實付) 依113年6月28日金管保壽字第11304207572號函修正\n4.特定處置費用保險金(實支實付)\n5.住院前後門診保險金\n(本保險商品為非保證續保之健康保險。)\n(本附約保險期間為一年，保險期間屆滿後不保證續保，且續保費率不保證永久不變，該續保費率若有調升\n或調降，將依報經主管機關核可後重新計算，請審慎投保。)\n(本附約「疾病」之等待期間為本附約生效日起持續有效30日之期間，但被保險人投保時保險年齡為零歲者，\n其遺傳性疾病之新生兒先天性代謝異常疾病檢查項目(以中央衛生主管機關公告為準)不受等待期間限制。)\n(受益人申領本附約第七條至第九條各項保險金時，應檢具醫療費用收據正本。)\n(各投保計劃別之每年保險金給付總限額請參附表一。)\n◎免費申訴電話：0800-213-269。\n【保險附約的構成】\n第一條\n本台灣人壽龍實在住院醫療健康保險附約（以下簡稱本附約）係依主保險契約（以下簡稱主契約）要保人之申請，\n並經本公司同意，附加於主契約訂定之。\n本保險單條款、附著之要保書、批註及其他約定書，均為本附約的構成部分。\n本附約的解釋，應探求附約當事人的真意，不得拘泥於所用的文字；如有疑義時，以作有利於被保險人的解釋為原\n則。\n【名詞定義】\n第二條\n本附約所稱名詞定義如下：\n一、「被保險人」係指主契約保險單內所載附加本附約之主契約被保險人。\n二、「疾病」：係指被保險人自本附約生效日起持續有效第31日起或自復效日起所發生之疾病。\n三、「傷害」：係指被保險人於本附約有效期間內，遭受意外傷害事故，因而蒙受之傷害。\n四、「意外傷害事故」：係指非由疾病引起之外來突發事故。\n五、「醫院」：係指依照醫療法規定領有開業執照並設有病房收治病人之公、私立及醫療法人醫院。\n六、「診所

In [40]:
from tools.graph_builder import TwlfGraphBuilder

graph_builder = TwlfGraphBuilder(graph, max_thread=5)
# chunks, root_graph_document, root_document, root_node = graph_builder.build_chunk_graph_with_parent_child(pages)
result = graph_builder.build_chunk_graph_with_parent_child(pages)
result.chunks

[{'chunk_id': '7ca17ea1-2ffb-4b3b-8234-d7c9973fc346',
  'chunk_doc': Document(metadata={'source': '台灣人壽龍實在住院醫療健康保險附約.pdf', 'total_page_num': 9}, page_content='台灣人壽保險股份有限公司\n（以下簡稱「本公司」）\n台灣人壽龍實在住院醫療健康保險附約\n主要給付項目： 中華民國113年7月1日\n1.住院日額保險金 台壽字第1132320033號函備查\n2.住院醫療費用保險金(實支實付) 中華民國113年10月1日\n3.門診手術費用保險金(實支實付) 依113年6月28日金管保壽字第11304207572號函修正\n4.特定處置費用保險金(實支實付)\n5.住院前後門診保險金\n(本保險商品為非保證續保之健康保險。)\n(本附約保險期間為一年，保險期間屆滿後不保證續保，且續保費率不保證永久不變，該續保費率若有調升\n或調降，將依報經主管機關核可後重新計算，請審慎投保')},
 {'chunk_id': 'dae95766-a9d5-4396-8b8c-c87605cf76c8',
  'chunk_doc': Document(metadata={'source': '台灣人壽龍實在住院醫療健康保險附約.pdf', 'total_page_num': 9}, page_content='。)\n(本附約「疾病」之等待期間為本附約生效日起持續有效30日之期間，但被保險人投保時保險年齡為零歲者，\n其遺傳性疾病之新生兒先天性代謝異常疾病檢查項目(以中央衛生主管機關公告為準)不受等待期間限制。)\n(受益人申領本附約第七條至第九條各項保險金時，應檢具醫療費用收據正本。)\n(各投保計劃別之每年保險金給付總限額請參附表一。)\n◎免費申訴電話：0800-213-269。')},
 {'chunk_id': '92983cff-8054-447e-b211-0e358fdf1d9d',
  'chunk_doc': Document(metadata={'source': '台灣人壽龍實在住院醫療健康保險附約.pdf', 'total_page_num': 9}, page_content='【保險附約的構成】\n第一

In [41]:
for detail in result.details:
    root_graph_document, root_document, root_node = detail.root_graph_document, detail.root_document, detail.root_node
    graph_builder.graph.add_graph_documents([root_graph_document])
    graph_builder.update_node_properties(root_node.id, root_document.metadata)

In [42]:
# from tools.TokenCounter import num_tokens_from_string

# tokens_num = 0
# for chunk in chunk_list:
#     doc = chunk['chunk_doc']
#     content = doc.page_content
#     curr_num = num_tokens_from_string(content)
#     tokens_num += num_tokens_from_string(content)
# tokens_num

In [43]:
graph_builder._get_combined_chunks(result.chunks)

[Document(metadata={'source': '台灣人壽龍實在住院醫療健康保險附約.pdf', 'total_page_num': 9, 'combined_chunk_ids': ['7ca17ea1-2ffb-4b3b-8234-d7c9973fc346']}, page_content='台灣人壽保險股份有限公司\n（以下簡稱「本公司」）\n台灣人壽龍實在住院醫療健康保險附約\n主要給付項目： 中華民國113年7月1日\n1.住院日額保險金 台壽字第1132320033號函備查\n2.住院醫療費用保險金(實支實付) 中華民國113年10月1日\n3.門診手術費用保險金(實支實付) 依113年6月28日金管保壽字第11304207572號函修正\n4.特定處置費用保險金(實支實付)\n5.住院前後門診保險金\n(本保險商品為非保證續保之健康保險。)\n(本附約保險期間為一年，保險期間屆滿後不保證續保，且續保費率不保證永久不變，該續保費率若有調升\n或調降，將依報經主管機關核可後重新計算，請審慎投保'),
 Document(metadata={'source': '台灣人壽龍實在住院醫療健康保險附約.pdf', 'total_page_num': 9, 'combined_chunk_ids': ['dae95766-a9d5-4396-8b8c-c87605cf76c8']}, page_content='。)\n(本附約「疾病」之等待期間為本附約生效日起持續有效30日之期間，但被保險人投保時保險年齡為零歲者，\n其遺傳性疾病之新生兒先天性代謝異常疾病檢查項目(以中央衛生主管機關公告為準)不受等待期間限制。)\n(受益人申領本附約第七條至第九條各項保險金時，應檢具醫療費用收據正本。)\n(各投保計劃別之每年保險金給付總限額請參附表一。)\n◎免費申訴電話：0800-213-269。'),
 Document(metadata={'source': '台灣人壽龍實在住院醫療健康保險附約.pdf', 'total_page_num': 9, 'combined_chunk_ids': ['92983cff-8054-447e-b211-0e358fdf1d9d']}, page_content='【保險附約的構成】\n第一條\n本台灣人壽龍實在住院醫

In [44]:

graph_documents = graph_builder.get_graph_from_llm(llm, result.chunks, allowedNodes=[], allowedRelationship=[])


100%|██████████| 122/122 [03:04<00:00,  1.51s/it]


In [45]:
graph_documents[0].source

Document(metadata={'source': '台灣人壽龍實在住院醫療健康保險附約.pdf', 'total_page_num': 9, 'combined_chunk_ids': ['e6bd9cf9-34bc-4f45-aefe-31a28b245c5c']}, page_content='【名詞定義】\n第二條\n本附約所稱名詞定義如下：\n一、「被保險人」係指主契約保險單內所載附加本附約之主契約被保險人。\n二、「疾病」：係指被保險人自本附約生效日起持續有效第31日起或自復效日起所發生之疾病。\n三、「傷害」：係指被保險人於本附約有效期間內，遭受意外傷害事故，因而蒙受之傷害。\n四、「意外傷害事故」：係指非由疾病引起之外來突發事故。\n五、「醫院」：係指依照醫療法規定領有開業執照並設有病房收治病人之公、私立及醫療法人醫院。\n六、「診所」：係指依照醫療法規定領有開業執照之診所。\n七、「醫師」：係指領有醫師證書與執業執照，合法執業者，且非要保人本人或被保險人本人')

In [46]:
graph.add_graph_documents(graph_documents, baseEntityLabel=True)

In [47]:
chunks_and_graphDocuments_list = graph_builder.get_chunk_and_graphDocument(graph_documents)
chunks_and_graphDocuments_list[0]

{'graph_doc': GraphDocument(nodes=[Node(id='被保險人', type='Concept', properties={'description': '主契約保險單內所載附加本附約之主契約被保險人'}), Node(id='疾病', type='Concept', properties={'description': '被保險人自本附約生效日起持續有效第31日起或自復效日起所發生之疾病'}), Node(id='傷害', type='Concept', properties={'description': '被保險人於本附約有效期間內，遭受意外傷害事故，因而蒙受之傷害'}), Node(id='意外傷害事故', type='Concept', properties={'description': '非由疾病引起之外來突發事故'}), Node(id='醫院', type='Concept', properties={'description': '依照醫療法規定領有開業執照並設有病房收治病人之公、私立及醫療法人醫院'}), Node(id='診所', type='Concept', properties={'description': '依照醫療法規定領有開業執照之診所'}), Node(id='醫師', type='Concept', properties={'description': '領有醫師證書與執業執照，合法執業者，且非要保人本人或被保險人本人'})], relationships=[], source=Document(metadata={'source': '台灣人壽龍實在住院醫療健康保險附約.pdf', 'total_page_num': 9, 'combined_chunk_ids': ['e6bd9cf9-34bc-4f45-aefe-31a28b245c5c'], 'id': 'bc6da031779886c864077156d2635afc'}, page_content='【名詞定義】\n第二條\n本附約所稱名詞定義如下：\n一、「被保險人」係指主契約保險單內所載附加本附約之主契約被保險人。\n二、「疾病」：係指被保險人自本附約生效日起持續有效第31日起或自復效日起所發生之疾病。\n三、「傷害」：係指

In [48]:
graph_builder.merge_relationship_between_chunk_and_entites(chunks_and_graphDocuments_list)